# install dependencies

In [ ]:
# !apt install default-jdk

In [ ]:
%pip install -r requirements.txt

In [2]:
!python -m spacy download en_core_web_trf

/home/arch/Projects/college/factcheck_en/dev_2/lib/python3.11/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 3.3 MB/s eta 0:00:0000:0100:03
  Obtaining dependency information for spacy-curated-transformers<0.3.0,>=0.2.0 from https://files.pythonhosted.org/packages/60/2e/a9e0d18db1fc63efd3111e9e05a02f4e491026ebbf57d8788f29e9045ca8/spacy_curated_transformers-0.2.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for curated-transformers<0.2.0,>=0.1.0 from https://files.pythonhosted.org/packages/d6/67/3b72b3fdfcadab61bc8f59c17e63770e526ffabd583ed32f174a7c01af85/curated_transformers-0.1.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for curated-tokenizers<0.1.0,>=0.0.7 from https://files.pythonhosted.org/packages/a4/fc/cfcf511db5658dcf1c8822de846ae97c1c955fb633b5e2d6d36b1f0fa441/curated_tokenizers-0.0.8-cp311-cp311-manylinux_

# download data and checkpoint

In [ ]:
!mkdir fever_wiki fever_wiki/corpora

# download data for retrieval
!curl -o ./fever_wiki/wiki-pages.zip https://fever.ai/download/fever/wiki-pages.zip
!unzip fever_wiki/wiki-pages.zip -d fever_wiki/

# download model checkpoints
!curl -o ./checkpoints/longformer_large_science.ckpt https://scifact.s3.us-west-2.amazonaws.com/longchecker/latest/checkpoints/longformer_large_science.ckpt &
!curl -o ./checkpoints/fever_sci.ckpt https://scifact.s3.us-west-2.amazonaws.com/longchecker/latest/checkpoints/fever_sci.ckpt &

In [ ]:
!python src/prepare_wiki.py

# index the corpus with pyserini

In [ ]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input fever_wiki/corpora \
  --index indexes/fever_wiki \
  --generator DefaultLuceneDocumentGenerator \
  --threads 8 \
  --storePositions --storeDocvectors --storeRaw

# start streamlit

In [4]:
!streamlit run Entry.py -- \
    --checkpoint_path checkpoints/fever_sci.ckpt \
    --index_dir indexes/fever_wiki \
    # --cuda \
    --batch_size 2


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.24.67.73:8501

/home/arch/Projects/college/factcheck_en/dev_2/lib/python3.11/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/home/arch/Projects/college/factcheck_en/dev_2/lib/python3.11/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/arch/Projects/college/factcheck_en/dev_2/lib/python3.11/site-packages/pytorch_lightning/utilities/migration/migration.py:207: You have multiple `ModelCheckpoint` callback states in this checkpoint, but we found state keys that would end up colliding with each other after an upgrade, which means we can't differentiate which of your checkpoint callbacks needs which states. At least one of your `ModelCheckpoint` callbacks will not be 